In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import io, base64
import joblib

# Load Boston Housing dataset
boston = fetch_openml(name="boston", version=1, as_frame=True)
X = boston.data
y = boston.target

#  Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA (keep enough components to explain ~95% variance)
pca = PCA(n_components=0.95)  # choose components that explain 95% variance
X_pca = pca.fit_transform(X_scaled)

print("Original shape:", X.shape)
print("Reduced shape after PCA:", X_pca.shape)
print("Explained variance ratio:", pca.explained_variance_ratio_)

#Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

#  Train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

#  Predictions
y_pred = model.predict(X_test)

#  Evaluation
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R² Score: {r2:.3f}")
print(f"Mean Squared Error: {mse:.3f}")


def plot_to_base64():
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    img_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    buffer.close()
    plt.close()  # close current figure
    return img_str
    
# VISUALIZATIONS 

# (A) Scree Plot
plt.figure(figsize=(6,4))
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("Scree Plot - PCA on Boston Housing")
plt.grid()
scree_plot = plot_to_base64()
plt.show()

# (B) Scatter plot
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y, cmap="viridis", edgecolor="k")
plt.colorbar(label="House Price ($1000s)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("Boston Housing - PCA Visualization")
scatter_plot = plot_to_base64()
plt.show()

# (C) Actual vs Predicted
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, edgecolor="k", alpha=0.7)
plt.plot([y.min(), y.max()], [y.min(), y.max()], "r--", lw=2)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual vs Predicted House Prices")
pred_plot = plot_to_base64()
plt.show()

# (D) Residuals Plot
residuals = y_test - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, edgecolor="k", alpha=0.7)
plt.axhline(y=0, color="r", linestyle="--")
plt.xlabel("Predicted Prices")
plt.ylabel("Residuals")
plt.title("Residuals Plot")
resid_plot = plot_to_base64()
plt.show()

pipeline = {
    'scaler': scaler,
    'pca': pca,
    'model': model,
    "mse": round(mse, 3),
    "r2": round(r2, 3),
    "plots": {
        "scree": scree_plot,
        "scatter": scatter_plot,
        "predicted": pred_plot,
        "residuals": resid_plot
    }
}
 
joblib.dump(pipeline, 'model.pkl')
print("Model saved to model.pkl")

Original shape: (506, 13)
Reduced shape after PCA: (506, 9)
Explained variance ratio: [0.47129606 0.11025193 0.0955859  0.06596732 0.06421661 0.05056978
 0.04118124 0.03046902 0.02130333]
R² Score: 0.598
Mean Squared Error: 29.510
Model saved to model.pkl


In [1]:
import joblib

data = joblib.load("model.pkl")
print(type(data))
print(data)


<class 'dict'>
{'scaler': StandardScaler(), 'pca': PCA(n_components=0.95), 'model': LinearRegression()}


In [11]:
import joblib

model_stuff = joblib.load("model.pkl")  # adjust path if needed
print(type(model_stuff))
print(model_stuff.keys())


<class 'dict'>
dict_keys(['scaler', 'pca', 'model', 'mse', 'r2', 'plots'])
